# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 27 2022 8:30AM,240448,10,8188330,"Herbivore Botanicals, LLC.",Executing
1,Jun 24 2022 4:06PM,240447,19,ADV80003558,"AdvaGen Pharma, Ltd",Released
2,Jun 24 2022 4:06PM,240447,19,ADV80003559,"AdvaGen Pharma, Ltd",Executing
3,Jun 24 2022 4:06PM,240447,19,ADV80003560,"AdvaGen Pharma, Ltd",Executing
4,Jun 24 2022 4:06PM,240447,19,ADV80003561,"AdvaGen Pharma, Ltd",Executing
5,Jun 24 2022 4:06PM,240447,19,ADV80003562,"AdvaGen Pharma, Ltd",Released
6,Jun 24 2022 4:06PM,240447,19,ADV80003563,"AdvaGen Pharma, Ltd",Executing
7,Jun 24 2022 4:06PM,240447,19,ADV80003564,"AdvaGen Pharma, Ltd",Executing
8,Jun 24 2022 4:06PM,240447,19,ADV80003565,"AdvaGen Pharma, Ltd",Released
9,Jun 24 2022 3:47PM,240446,19,60012010,"GUSA Granules USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
7,240442,Released,5
8,240446,Released,1
9,240447,Executing,5
10,240447,Released,3
11,240448,Executing,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
240429,4.0,1.0
240442,NaN,5.0
240446,NaN,1.0
240447,5.0,3.0
240448,1.0,NaN


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
240386,0.0,36.0
240423,0.0,20.0
240424,0.0,54.0
240427,21.0,1.0
240429,4.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
240386,0,36
240423,0,20
240424,0,54
240427,21,1
240429,4,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,240386,0,36
1,240423,0,20
2,240424,0,54
3,240427,21,1
4,240429,4,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,240386,,36
1,240423,,20
2,240424,,54
3,240427,21,1
4,240429,4,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 27 2022 8:30AM,240448,10,"Herbivore Botanicals, LLC."
1,Jun 24 2022 4:06PM,240447,19,"AdvaGen Pharma, Ltd"
9,Jun 24 2022 3:47PM,240446,19,"GUSA Granules USA, Inc."
10,Jun 24 2022 3:39PM,240442,19,"GUSA Granules USA, Inc."
15,Jun 24 2022 3:00PM,240424,10,ISDIN Corporation
69,Jun 24 2022 2:59PM,240423,20,Alumier Labs Inc.
89,Jun 24 2022 2:56PM,240427,10,ISDIN Corporation
111,Jun 24 2022 2:54PM,240429,10,ISDIN Corporation
116,Jun 24 2022 9:22AM,240386,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jun 27 2022 8:30AM,240448,10,"Herbivore Botanicals, LLC.",1,
1,Jun 24 2022 4:06PM,240447,19,"AdvaGen Pharma, Ltd",5,3
2,Jun 24 2022 3:47PM,240446,19,"GUSA Granules USA, Inc.",,1
3,Jun 24 2022 3:39PM,240442,19,"GUSA Granules USA, Inc.",,5
4,Jun 24 2022 3:00PM,240424,10,ISDIN Corporation,,54
5,Jun 24 2022 2:59PM,240423,20,Alumier Labs Inc.,,20
6,Jun 24 2022 2:56PM,240427,10,ISDIN Corporation,21,1
7,Jun 24 2022 2:54PM,240429,10,ISDIN Corporation,4,1
8,Jun 24 2022 9:22AM,240386,10,ISDIN Corporation,,36


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 27 2022 8:30AM,240448,10,"Herbivore Botanicals, LLC.",,1
1,Jun 24 2022 4:06PM,240447,19,"AdvaGen Pharma, Ltd",3,5
2,Jun 24 2022 3:47PM,240446,19,"GUSA Granules USA, Inc.",1,
3,Jun 24 2022 3:39PM,240442,19,"GUSA Granules USA, Inc.",5,
4,Jun 24 2022 3:00PM,240424,10,ISDIN Corporation,54,
5,Jun 24 2022 2:59PM,240423,20,Alumier Labs Inc.,20,
6,Jun 24 2022 2:56PM,240427,10,ISDIN Corporation,1,21
7,Jun 24 2022 2:54PM,240429,10,ISDIN Corporation,1,4
8,Jun 24 2022 9:22AM,240386,10,ISDIN Corporation,36,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 27 2022 8:30AM,240448,10,"Herbivore Botanicals, LLC.",,1
1,Jun 24 2022 4:06PM,240447,19,"AdvaGen Pharma, Ltd",3,5
2,Jun 24 2022 3:47PM,240446,19,"GUSA Granules USA, Inc.",1,
3,Jun 24 2022 3:39PM,240442,19,"GUSA Granules USA, Inc.",5,
4,Jun 24 2022 3:00PM,240424,10,ISDIN Corporation,54,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jun 27 2022 8:30AM,240448,10,"Herbivore Botanicals, LLC.",NaN,1.0
1,Jun 24 2022 4:06PM,240447,19,"AdvaGen Pharma, Ltd",3.0,5.0
2,Jun 24 2022 3:47PM,240446,19,"GUSA Granules USA, Inc.",1.0,NaN
3,Jun 24 2022 3:39PM,240442,19,"GUSA Granules USA, Inc.",5.0,NaN
4,Jun 24 2022 3:00PM,240424,10,ISDIN Corporation,54.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 27 2022 8:30AM,240448,10,"Herbivore Botanicals, LLC.",0.0,1.0
1,Jun 24 2022 4:06PM,240447,19,"AdvaGen Pharma, Ltd",3.0,5.0
2,Jun 24 2022 3:47PM,240446,19,"GUSA Granules USA, Inc.",1.0,0.0
3,Jun 24 2022 3:39PM,240442,19,"GUSA Granules USA, Inc.",5.0,0.0
4,Jun 24 2022 3:00PM,240424,10,ISDIN Corporation,54.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1202114,92.0,26.0
19,721335,9.0,5.0
20,240423,20.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1202114,92.0,26.0
1,19,721335,9.0,5.0
2,20,240423,20.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,92.0,26.0
1,19,9.0,5.0
2,20,20.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,92.0
1,19,Released,9.0
2,20,Released,20.0
3,10,Executing,26.0
4,19,Executing,5.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,19,20
Status,,,
Executing,26.0,5.0,0.0
Released,92.0,9.0,20.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,19,20
0,Executing,26.0,5.0,0.0
1,Released,92.0,9.0,20.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,19,20
0,Executing,26.0,5.0,0.0
1,Released,92.0,9.0,20.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()